# Advanced OpenAI Function Calling

This notebook demonstrates advanced usage of OpenAI's function calling API, including:
- Multiple functions
- Nested function calls
- Parallel function calls
- Function chaining

## Setup

First, let's install and import the required packages.

In [1]:
!pip install openai
from openai import OpenAI
import json
from typing import List, Dict, Any
from datetime import datetime, timedelta

# Initialize the client
client = OpenAI(api_key="sfakeee")

## 1. Multiple Functions Example

Let's create a task management system with multiple functions for managing tasks and categories.

In [2]:
# Mock database
tasks_db = {
    "tasks": [],
    "categories": ["Work", "Personal", "Shopping", "Health"]
}

def add_task(title: str, category: str, due_date: str, priority: str = "medium") -> str:
    """Add a new task to the system"""
    if category not in tasks_db["categories"]:
        raise ValueError(f"Invalid category: {category}")

    try:
        due = datetime.strptime(due_date, "%Y-%m-%d").date()
    except ValueError:
        raise ValueError("Invalid date format. Use YYYY-MM-DD")

    if priority not in ["low", "medium", "high"]:
        raise ValueError("Priority must be low, medium, or high")

    task = {
        "id": len(tasks_db["tasks"]) + 1,
        "title": title,
        "category": category,
        "due_date": due_date,
        "priority": priority,
        "completed": False
    }
    tasks_db["tasks"].append(task)
    return f"Task '{title}' added successfully with ID {task['id']}"

def list_tasks(category: str = None, priority: str = None) -> str:
    """List tasks with optional filtering"""
    tasks = tasks_db["tasks"]

    if category:
        if category not in tasks_db["categories"]:
            raise ValueError(f"Invalid category: {category}")
        tasks = [t for t in tasks if t["category"] == category]

    if priority:
        if priority not in ["low", "medium", "high"]:
            raise ValueError("Priority must be low, medium, or high")
        tasks = [t for t in tasks if t["priority"] == priority]

    if not tasks:
        return "No tasks found"

    result = "Tasks:\n"
    for task in tasks:
        status = "✓" if task["completed"] else " "
        result += f"[{status}] {task['id']}. {task['title']} ({task['category']}) - {task['priority']} priority, due {task['due_date']}\n"
    return result

def complete_task(task_id: int) -> str:
    """Mark a task as completed"""
    for task in tasks_db["tasks"]:
        if task["id"] == task_id:
            if task["completed"]:
                return f"Task {task_id} is already completed"
            task["completed"] = True
            return f"Task {task_id} marked as completed"
    raise ValueError(f"Task {task_id} not found")

# Define function schemas
task_functions = [
    {
        "type": "function",
        "function": {
            "name": "add_task",
            "description": "Add a new task to the system",
            "parameters": {
                "type": "object",
                "properties": {
                    "title": {
                        "type": "string",
                        "description": "The title of the task"
                    },
                    "category": {
                        "type": "string",
                        "enum": ["Work", "Personal", "Shopping", "Health"],
                        "description": "The category of the task"
                    },
                    "due_date": {
                        "type": "string",
                        "description": "The due date in YYYY-MM-DD format"
                    },
                    "priority": {
                        "type": "string",
                        "enum": ["low", "medium", "high"],
                        "description": "The priority level of the task"
                    }
                },
                "required": ["title", "category", "due_date"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "list_tasks",
            "description": "List tasks with optional filtering",
            "parameters": {
                "type": "object",
                "properties": {
                    "category": {
                        "type": "string",
                        "enum": ["Work", "Personal", "Shopping", "Health"],
                        "description": "Filter tasks by category"
                    },
                    "priority": {
                        "type": "string",
                        "enum": ["low", "medium", "high"],
                        "description": "Filter tasks by priority"
                    }
                }
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "complete_task",
            "description": "Mark a task as completed",
            "parameters": {
                "type": "object",
                "properties": {
                    "task_id": {
                        "type": "integer",
                        "description": "The ID of the task to mark as completed"
                    }
                },
                "required": ["task_id"]
            }
        }
    }
]

def process_task_command(user_input: str) -> str:
    """Process task-related commands using multiple functions"""
    # Get model's response with function calls
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": user_input}],
        tools=task_functions,
        tool_choice="auto"
    )

    # Extract function call
    message = response.choices[0].message
    tool_calls = message.tool_calls

    if tool_calls:
        # Process the function call
        function_call = tool_calls[0]
        function_args = json.loads(function_call.function.arguments)

        # Call the appropriate function
        try:
            if function_call.function.name == "add_task":
                return add_task(**function_args)
            elif function_call.function.name == "list_tasks":
                return list_tasks(**function_args)
            elif function_call.function.name == "complete_task":
                return complete_task(**function_args)
        except ValueError as e:
            return f"Error: {str(e)}"

    return "I couldn't process that command"

# Test the task management system
test_commands = [
    "Add a task to buy groceries in the Shopping category, due tomorrow, high priority",
    "Add a work task for the quarterly report, due next week, medium priority",
    "Show me all shopping tasks",
    "Mark task 1 as completed",
    "List all high priority tasks",
    "Show me all tasks"
]

for command in test_commands:
    print(f"Command: {command}")
    result = process_task_command(command)
    print(f"Result: {result}")
    print("-"*50)

Command: Add a task to buy groceries in the Shopping category, due tomorrow, high priority
Result: Task 'Buy groceries' added successfully with ID 1
--------------------------------------------------
Command: Add a work task for the quarterly report, due next week, medium priority
Result: Task 'Prepare quarterly report' added successfully with ID 2
--------------------------------------------------
Command: Show me all shopping tasks
Result: Tasks:
[ ] 1. Buy groceries (Shopping) - high priority, due 2023-10-05

--------------------------------------------------
Command: Mark task 1 as completed
Result: Task 1 marked as completed
--------------------------------------------------
Command: List all high priority tasks
Result: Tasks:
[✓] 1. Buy groceries (Shopping) - high priority, due 2023-10-05

--------------------------------------------------
Command: Show me all tasks
Result: Tasks:
[✓] 1. Buy groceries (Shopping) - high priority, due 2023-10-05
[ ] 2. Prepare quarterly report (Wor

## 2. Function Chaining Example

Let's create a more complex example that chains multiple function calls together for a travel planning system.

In [3]:
# Mock travel data
travel_db = {
    "flights": [
        {"id": "FL001", "from": "New York", "to": "London", "price": 500, "date": "2024-05-15"},
        {"id": "FL002", "from": "London", "to": "Paris", "price": 200, "date": "2024-05-16"},
        {"id": "FL003", "from": "Paris", "to": "Rome", "price": 150, "date": "2024-05-17"}
    ],
    "hotels": [
        {"id": "HT001", "city": "London", "name": "Luxury Hotel", "price": 300},
        {"id": "HT002", "city": "Paris", "name": "Boutique Hotel", "price": 200},
        {"id": "HT003", "city": "Rome", "name": "Grand Hotel", "price": 250}
    ],
    "activities": [
        {"id": "AC001", "city": "London", "name": "Tower Tour", "price": 50},
        {"id": "AC002", "city": "Paris", "name": "Museum Pass", "price": 40},
        {"id": "AC003", "city": "Rome", "name": "Colosseum Tour", "price": 45}
    ]
}

def search_flights(origin: str, destination: str, date: str) -> str:
    """Search for available flights"""
    flights = [f for f in travel_db["flights"]
              if f["from"].lower() == origin.lower()
              and f["to"].lower() == destination.lower()
              and f["date"] == date]

    if not flights:
        return f"No flights found from {origin} to {destination} on {date}"

    result = "Available flights:\n"
    for flight in flights:
        result += f"Flight {flight['id']}: {flight['from']} to {flight['to']}, ${flight['price']} on {flight['date']}\n"
    return result

def search_hotels(city: str) -> str:
    """Search for available hotels"""
    hotels = [h for h in travel_db["hotels"] if h["city"].lower() == city.lower()]

    if not hotels:
        return f"No hotels found in {city}"

    result = f"Hotels in {city}:\n"
    for hotel in hotels:
        result += f"Hotel {hotel['id']}: {hotel['name']}, ${hotel['price']} per night\n"
    return result

def search_activities(city: str) -> str:
    """Search for available activities"""
    activities = [a for a in travel_db["activities"] if a["city"].lower() == city.lower()]

    if not activities:
        return f"No activities found in {city}"

    result = f"Activities in {city}:\n"
    for activity in activities:
        result += f"Activity {activity['id']}: {activity['name']}, ${activity['price']}\n"
    return result

# Define function schemas
travel_functions = [
    {
        "type": "function",
        "function": {
            "name": "search_flights",
            "description": "Search for available flights",
            "parameters": {
                "type": "object",
                "properties": {
                    "origin": {
                        "type": "string",
                        "description": "The departure city"
                    },
                    "destination": {
                        "type": "string",
                        "description": "The arrival city"
                    },
                    "date": {
                        "type": "string",
                        "description": "The travel date in YYYY-MM-DD format"
                    }
                },
                "required": ["origin", "destination", "date"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "search_hotels",
            "description": "Search for available hotels",
            "parameters": {
                "type": "object",
                "properties": {
                    "city": {
                        "type": "string",
                        "description": "The city to search hotels in"
                    }
                },
                "required": ["city"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "search_activities",
            "description": "Search for available activities",
            "parameters": {
                "type": "object",
                "properties": {
                    "city": {
                        "type": "string",
                        "description": "The city to search activities in"
                    }
                },
                "required": ["city"]
            }
        }
    }
]

def plan_trip(user_input: str) -> str:
    """Process travel planning requests using function chaining"""
    # Initialize conversation
    messages = [{"role": "user", "content": user_input}]

    # First response to understand the request
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=messages,
        tools=travel_functions,
        tool_choice="auto"
    )

    result = ""
    for i in range(3):  # Allow up to 3 function calls in chain
        message = response.choices[0].message
        messages.append({"role": "assistant", "content": message.content or ""})

        tool_calls = message.tool_calls
        if not tool_calls:
            break

        # Process each function call
        for tool_call in tool_calls:
            function_name = tool_call.function.name
            function_args = json.loads(tool_call.function.arguments)

            # Call the appropriate function
            try:
                if function_name == "search_flights":
                    function_response = search_flights(**function_args)
                elif function_name == "search_hotels":
                    function_response = search_hotels(**function_args)
                elif function_name == "search_activities":
                    function_response = search_activities(**function_args)
                else:
                    continue

                result += function_response + "\n"
                messages.append({
                    "role": "function",
                    "name": function_name,
                    "content": function_response
                })
            except ValueError as e:
                result += f"Error: {str(e)}\n"

        # Get next response in chain
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=messages,
            tools=travel_functions,
            tool_choice="auto"
        )

    return result.strip()

# Test the travel planning system
test_queries = [
    "I want to travel from New York to London on May 15, 2024. Find me flights and hotels.",
    "What activities are available in Paris and how much do they cost?",
    "Plan a trip from New York to Rome via London and Paris for May 2024"
]

for query in test_queries:
    print(f"Query: {query}")
    result = plan_trip(query)
    print(f"Result:\n{result}")
    print("-"*50)

Query: I want to travel from New York to London on May 15, 2024. Find me flights and hotels.
Result:
Available flights:
Flight FL001: New York to London, $500 on 2024-05-15

Hotels in London:
Hotel HT001: Luxury Hotel, $300 per night
--------------------------------------------------
Query: What activities are available in Paris and how much do they cost?
Result:
Activities in Paris:
Activity AC002: Museum Pass, $40

Activities in Paris:
Activity AC002: Museum Pass, $40
--------------------------------------------------
Query: Plan a trip from New York to Rome via London and Paris for May 2024
Result:
No flights found from New York to London on 2024-05-01
No flights found from New York to London on 2024-05-01
--------------------------------------------------


## Key Points

1. Multiple Functions:
   - Define multiple related functions in a system
   - Let the model choose which function to call
   - Handle different types of operations

2. Function Chaining:
   - Pass function results back to the model
   - Allow the model to make multiple function calls
   - Build complex workflows

3. Best Practices:
   - Keep function schemas clear and specific
   - Handle errors gracefully
   - Maintain conversation context
   - Limit chain length to prevent infinite loops

4. Advanced Features:
   - Parallel function calls
   - Nested function calls
   - State management
   - Complex workflows